# Example 2 - Custom Approach

This notebook demonstrates how to use the TX2 dashboard with a sequence classification transformer using the custom approach as described in the "Basic Usage" docs. To demonstrate, we simply take the default functions for each and manually define them.

In [1]:
%cd -q ..
%load_ext autoreload
%autoreload 2
%matplotlib inline

We enable logging to view the output from `wrapper.prepare()` further down in the notebook. (It's a long running function, and logs which step it's on.)

In [2]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from torch import cuda
from transformers import AutoModel, AutoTokenizer, BertTokenizer
import pandas as pd

In this example notebook, we use the 20 newsgroups dataset, which can be downloaded through sklearn via below:

In [4]:
from sklearn.datasets import fetch_20newsgroups

train_data = fetch_20newsgroups(subset='train')
test_data = fetch_20newsgroups(subset='test')

device = 'cuda' if cuda.is_available() else 'cpu'

Defined below is a simple sequence classification model with a variable for the language model itself `l1`. Since it is a BERT model, we take the sequence embedding from the `[CLS]` token (via `output_1[0][:, 0, :])`) and pipe that into the linear layer.

In [5]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.l1 = AutoModel.from_pretrained("bert-base-cased")
        self.l2 = torch.nn.Linear(768, 20)

    def forward(self, ids, mask):
        output_1= self.l1(ids, mask)
        output = self.l2(output_1[0][:, 0, :]) # use just the [CLS] output embedding
        return output
    
model = BERTClass()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Some simplistic data cleaning, and putting all data into dataframes for the wrapper

In [6]:
def clean_text(text):
    text = text[text.index("\n\n")+2:]
    text = text.replace("\n", " ")
    text = text.replace("    ", " ")
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.strip()
    return text

In [7]:
train_rows = []
for i in range(len(train_data["data"])):
    row = {}
    row["text"] = clean_text(train_data["data"][i])
    row["target"] = train_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    train_rows.append(row)
train_df = pd.DataFrame(train_rows)

In [8]:
test_rows = []
for i in range(len(test_data["data"])):
    row = {}
    row["text"] = clean_text(test_data["data"][i])
    row["target"] = test_data['target'][i]
    if row["text"] == "" or row["text"] == " ": continue
    test_rows.append(row)
test_df = pd.DataFrame(test_rows)

## Training

This section minimally trains the classification and language model - nothing fancy here, just to give the dashboard demo something to work with. Most of this is similar to the huggingface tutorial notebooks.

In [9]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)

In [10]:
class EncodedSet(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        print(self.len)

    def __getitem__(self, index):
        text = str(self.data.text[index])
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'targets': torch.tensor(self.data.target[index], dtype=torch.long)
        }

    def __len__(self):
        return self.len
    
train_set = EncodedSet(train_df, tokenizer, 256)
test_set = EncodedSet(test_df[:1000], tokenizer, 256)

train_params = {'batch_size': 16,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 2,
                'shuffle': True,
                'num_workers': 0
                }

# put everything into data loaders
train_loader = DataLoader(train_set, **train_params)
test_loader = DataLoader(test_set, **test_params)

11296
1000


In [11]:
def train(epoch):
    model.train()

    
    loss_history = []
    for _,data in tqdm(enumerate(train_loader, 0), total=len(train_loader), desc=f"Epoch {epoch}"):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask).squeeze()

        optimizer.zero_grad()
        loss = loss_function(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        loss_history.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         torch.cuda.empty_cache()
    return loss_history

In [12]:
losses = []
for epoch in range(1):
    losses.extend(train(epoch))

/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  3.0621426105499268
Epoch: 0, Loss:  2.1853384971618652
Epoch: 0, Loss:  1.2323518991470337
Epoch: 0, Loss:  0.7899547815322876
Epoch: 0, Loss:  0.6378146409988403
Epoch: 0, Loss:  0.34727099537849426
Epoch: 0, Loss:  0.7990703582763672
Epoch: 0, Loss:  0.4353039562702179



The wrapper uses an `encodings` dictionary for various labels/visualizations, and can be set up with something similar to:

In [13]:
encodings = {}
for index, entry in enumerate(train_data["target_names"]):
    encodings[entry] = index
encodings

{'alt.atheism': 0,
 'comp.graphics': 1,
 'comp.os.ms-windows.misc': 2,
 'comp.sys.ibm.pc.hardware': 3,
 'comp.sys.mac.hardware': 4,
 'comp.windows.x': 5,
 'misc.forsale': 6,
 'rec.autos': 7,
 'rec.motorcycles': 8,
 'rec.sport.baseball': 9,
 'rec.sport.hockey': 10,
 'sci.crypt': 11,
 'sci.electronics': 12,
 'sci.med': 13,
 'sci.space': 14,
 'soc.religion.christian': 15,
 'talk.politics.guns': 16,
 'talk.politics.mideast': 17,
 'talk.politics.misc': 18,
 'talk.religion.misc': 19}

## TX2

This section shows how to put everything into the TX2 wrapper to get the dashboard widget displayed.

In [14]:
from tx2.wrapper import Wrapper
from tx2.dashboard import Dashboard

/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/llvmlite/binding/ffi.py:137: UserWarning: Module tx2 was already imported from None, but /home/81n/lab/tx2 is being added to sys.path
  from pkg_resources import resource_filename


In [15]:
from tx2 import utils

def custom_encoding_function(text):
    encoded = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=256,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=True,
    )
    return {
        "input_ids": torch.tensor(encoded["input_ids"], device=device),
        "attention_mask": torch.tensor(encoded["attention_mask"], device=device),
    }

def custom_classification_function(inputs):
    return torch.argmax(model(inputs["input_ids"], inputs["attention_mask"]), dim=1)

def custom_embedding_function(inputs):
    return model.l1(inputs["input_ids"], inputs["attention_mask"])[0][:, 0, :]  # [CLS] token embedding

def custom_soft_classification_function(inputs):
    return model(inputs["input_ids"], inputs["attention_mask"])


In [16]:
wrapper = Wrapper(
    train_texts=train_df.text,
    train_labels=train_df.target,
    test_texts=test_df.text[:2000],
    test_labels=test_df.target[:2000],
    encodings=encodings, 
    cache_path="data/custom_cache",
    overwrite=False
)
wrapper.encode_function = custom_encoding_function
wrapper.classification_function = custom_classification_function
wrapper.soft_classification_function = custom_soft_classification_function
wrapper.embedding_function = custom_embedding_function
wrapper.prepare()

INFO:root:Cache path found
INFO:root:Checking for cached predictions...
INFO:root:cached version 'data/custom_cache/predictions.json' found
INFO:root:Checking for cached embeddings...
INFO:root:cached version 'data/custom_cache/embedding_training.json' found
INFO:root:cached version 'data/custom_cache/embedding_testing.json' found
INFO:root:Checking for cached projections...
INFO:root:cached version 'data/custom_cache/projections_training.json' found
INFO:root:cached version 'data/custom_cache/projections_testing.json' found
INFO:root:cached version 'data/custom_cache/projector.pkl.gz' found
INFO:root:Checking for cached salience maps...
INFO:root:cached version 'data/custom_cache/salience.pkl.gz' found
INFO:root:Checking for cached cluster profiles...
INFO:root:cached version 'data/custom_cache/cluster_profiles.pkl.gz' found
INFO:root:cached version 'data/custom_cache/clusters.json' found
INFO:root:cached version 'data/custom_cache/cluster_labels.json' found
INFO:root:cached version '

In [17]:
%matplotlib agg
dash = Dashboard(wrapper, show_wordclouds=True)
dash.render()

/home/81n/miniconda3/envs/tx2/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


To play with different UMAP and DBSCAN arguments without having to recompute the entire `prepare()` function, you can use `recompute_projections` (which will recompute both the projections and visual clusterings) or `recompute_visual_clusterings` (which will only redo the clustering)

In [18]:
# wrapper.recompute_visual_clusterings(dbscan_args=dict())
# wrapper.recompute_projections(umap_args=dict(min_dist=.2), dbscan_args=dict())

To test or debug the classification model/see what raw outputs the viusualizations are getting, or create your own visualization tools, you can manually call the `classify()`, `soft_classify()`, `embed()` functions, or get access to any of the cached data as seen in the bottom cell

In [19]:
wrapper.classify(["testing"])

[6]

In [20]:
wrapper.soft_classify(["testing"])

[[-0.0289225485175848,
  0.21903502941131592,
  -0.18919363617897034,
  0.1605398803949356,
  0.35047945380210876,
  -0.5973342657089233,
  1.0397224426269531,
  1.237997055053711,
  1.0223766565322876,
  -0.4297390282154083,
  -0.6682910919189453,
  -1.2943071126937866,
  1.389951229095459,
  1.2935385704040527,
  1.6164648532867432,
  -1.0083789825439453,
  -0.5446566343307495,
  -0.4000367820262909,
  0.32286688685417175,
  -0.2159416824579239]]

In [21]:
wrapper.embed(["testing"])

[[0.5794275999069214,
  -0.30842527747154236,
  -0.0424782894551754,
  -0.30981895327568054,
  -0.950285792350769,
  -0.6514168381690979,
  -0.3009045720100403,
  -0.32175737619400024,
  -1.2961032390594482,
  -1.067375898361206,
  0.5512518286705017,
  -0.06799433380365372,
  -0.5763489007949829,
  0.9716912508010864,
  -0.12169316411018372,
  -0.40487140417099,
  0.12381374090909958,
  -0.1836201548576355,
  0.8955320119857788,
  0.30653151869773865,
  0.038127750158309937,
  1.0368579626083374,
  0.5208004713058472,
  -0.216702401638031,
  -0.46597930788993835,
  -0.24625352025032043,
  0.20494405925273895,
  0.43428710103034973,
  -0.13562679290771484,
  -0.06501410901546478,
  -0.8096764087677002,
  1.6236478090286255,
  -0.5914164185523987,
  -0.09828721731901169,
  -0.04803937301039696,
  0.4422769546508789,
  -0.023149991407990456,
  -0.13874715566635132,
  0.2850764989852905,
  0.30533310770988464,
  -1.0567861795425415,
  -0.04981256648898125,
  -0.6363614201545715,
  -0.1231

In [22]:
# cached data:
# wrapper.embeddings_training
# wrapper.embeddings_testing
# wrapper.projector
# wrapper.projections_training
# wrapper.projections_testing
# wrapper.salience_maps
# wrapper.clusters
# wrapper.cluster_profiles
# wrapper.cluster_words_freqs
# wrapper.cluster_class_word_sets